In [1]:
%loadlibs
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from utils import rnmse

Loaded libraries:
	- numpy (np)
	- matplotlib.pyplot (plt)
	- torch
	- torch.nn (nn)
	- torch.optim (optim)
	- tqdm


In [2]:
train_size=27000
test_size=1000

In [37]:
x = torch.cat([torch.load("dataset/x_train.pt"), torch.load("dataset/x_train_2.pt")])
y = torch.cat([torch.load("dataset/y_train.pt"), torch.load("dataset/y_train_2.pt")])
c = torch.cat([torch.load("dataset/c_train.pt"), torch.load("dataset/c_train_2.pt")])

x_test = torch.load("dataset/x_test.pt")
y_test = torch.load("dataset/y_test.pt")
c_test = torch.load("dataset/c_test.pt")

#x_min, x_max, c_min, c_max, y_min, y_max = x[:train_size].min(), x[:train_size].max(), c[:train_size].min(), c[:train_size].max(), y[:train_size].min(), y[:train_size].max()

#x = (x-x_min)/(x_max-x_min)
#c = (c-c_min)/(c_max-c_min)
#y = (y-y_min)/(y_max-y_min)

#x_test = (x_test-x_min)/(x_max-x_min)
#c_test = (c_test-c_min)/(c_max-c_min)
#y_test = (y_test-y_min)/(y_max-y_min)

In [38]:
x, y, c, x_test, y_test, c_test = x.numpy(), y.numpy(), c.numpy(), x_test.numpy(), y_test.numpy(), c_test.numpy()

In [39]:
seismogram_shape = x.shape[1:]

In [40]:
x = x.reshape((x.shape[0], -1))
y = y.reshape((y.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))
y_test = y_test.reshape((y_test.shape[0], -1))

In [48]:
pca1 = PCA(n_components=1024)
pca2 = PCA(n_components=1024)

In [49]:
pca1 = pca1.fit(x)

In [50]:
pca2 = pca2.fit(y)

In [51]:
x_pca, y_pca = pca1.transform(x), pca2.transform(y)

In [52]:
lm = LinearRegression()
lm = lm.fit(x_pca, y_pca)

In [53]:
y_hat = pca2.inverse_transform(lm.predict(pca1.transform(x_test)))
y_test, y_hat = y_test.reshape((-1, seismogram_shape[0], seismogram_shape[1])), y_hat.reshape((-1, seismogram_shape[0], seismogram_shape[1]))

In [54]:
rnmse(torch.tensor(y_hat), torch.tensor(y_test))

tensor(0.0884)

In [34]:
class CorrectionMLP(nn.Module):
    def __init__(self, nx, hidden=64):
        super().__init__()
        self.c_encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(nx * nx, hidden//2),
            nn.Tanh(),
            nn.Linear(hidden//2, hidden),
            nn.Tanh(),
        )
        self.xt_block = nn.Sequential(
            nn.Linear(nx + hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden//2),
            nn.Tanh(),
            nn.Linear(hidden//2, nx),
        )
        self.correction= nn.Sequential(
            nn.Linear(nx, hidden),
            nn.Tanh(),
            nn.Linear(hidden, nx),
        )

    def forward(self, x, c):
        n, nt, nx = x.shape

        c_latent = self.c_encoder(c)
        c_latent = c_latent[:, None, :].expand(n, nt, -1)

        inp = torch.cat([x, c_latent], dim=-1)
        y = self.xt_block(inp)
        y = self.correction(y)
        return y + x
model = torch.load("saved_models/correction_cnn.pt", weights_only=False)

In [35]:
x_test, y_test, c_test = torch.tensor(x_test), torch.tensor(y_test), torch.tensor(c_test)

/var/folders/g7/l9_hhr256md6lt7yldvmnjjw0000gn/T/ipykernel_17379/2696595761.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test, y_test, c_test = torch.tensor(x_test), torch.tensor(y_test), torch.tensor(c_test)


In [36]:
x_test = x_test*(x_max-x_min)+ x_min
y_test = y_test*(y_max-y_min)+ y_min
c_test = c_test*(c_max-c_min)+ c_min
rnmse(model(torch.tensor(x_test.reshape((-1, *seismogram_shape))).to('mps'), torch.tensor(c_test).to('mps')), torch.tensor(y_test).to('mps'))

/var/folders/g7/l9_hhr256md6lt7yldvmnjjw0000gn/T/ipykernel_17379/3709978188.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rnmse(model(torch.tensor(x_test.reshape((-1, *seismogram_shape))).to('mps'), torch.tensor(c_test).to('mps')), torch.tensor(y_test).to('mps'))


tensor(0.3947, device='mps:0', grad_fn=<PowBackward0>)